# Scrapping Dados - Lutadores MMA

Data coleta: 14/08/2020 - 23:45

Este processo realiza o scrapping dos dados das seguintes fontes:

- Pesquisa no Google com informações da página http://sherdog.com
- Site Tapology: https://www.tapology.com/

Os sites são utilizados para obter os seguintes dados:

- **Peso**
- **Altura**
- **Data de aniversário**
- **Alcance dos golpes**

In [41]:
#import urllib.request as urlRequest
#import urllib.parse as urlParse
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import json

In [42]:
headers = {
    'DNT': '1',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'es-ES,es;q=0.8,en;q=0.6',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'http://www.it-ebooks.info/',
    'Connection': 'keep-alive',
}


In [43]:
df_fighters = pd.read_csv('datasets_255092_536001_raw_fighter_details.csv')

#Verifica se ha algum campo nulo - 1748  de 3312
df_missing_values = df_fighters.query('Height != Height or Weight != Weight or Reach != Reach or DOB != DOB')

In [ ]:
for index, row in df_missing_values.iterrows():
    #if index < 1:
        
    birth_date = None
    height = None
    weight = None
    reach = None

    birth_date_sherdog = None
    height_sherdog = None
    weight_sherdog = None

    birth_date_tapology = None
    weight_tapology = None
    height_tapology = None
    reach_tapology = None


    first_result = ""
    tapology_url = ""
    
    first_exception = ""

    fighter_name = row['fighter_name'].replace(" ", "+")
    print(fighter_name)

    try:

        ##### first: search in sherdog.com
        try:
            r = requests.get('https://www.google.com/search?q=sherdog.com+'+fighter_name, headers=headers)

            soup = BeautifulSoup(r.text, 'html.parser')
            first_result = soup.find("div", {"class":"r"}).find("a").get("href")

            r = requests.get(first_result, headers=headers)

            soup = BeautifulSoup(r.text, 'html.parser')

            birth_date_sherdog = soup.find("span", {"itemprop":"birthDate"})
            height_sherdog = soup.find("strong", {"itemprop":"height"})
            weight_sherdog = soup.find("strong", {"itemprop":"weight"})

            if(birth_date_sherdog is not None):
                birth_date_sherdog = birth_date_sherdog.text
                #print(birth_date_sherdog)
            if(height_sherdog is not None):
                height_sherdog = height_sherdog.text
                #print(height_sherdog)
            if(weight_sherdog is not None):
                weight_sherdog = weight_sherdog.text + '.'
                #print(weight_sherdog)
            
            
        except Exception as e:
            first_exception = str(e)

        ##### second: search in tapology.com
        tapology_url = 'https://www.tapology.com/search?commit=Enviar&model%5Bfighters%5D=fightersSearch&term=' + fighter_name

        r = requests.get(tapology_url, headers=headers, timeout=20)
        soup = BeautifulSoup(r.text, 'html.parser')
        search_results = soup.find("div", {"id":"content"}).find("h3").text

        chars_to_remove = ['(', ')']
        rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
        number_of_results = re.sub(rx, '', search_results.split()[2])

        if int(number_of_results) == 1:
            search_result_url = soup.find("div", 
                                          {"id":"content"}).find("table", 
                                                                 {"class":"fcLeaderboard"}).find("td", 
                                                                                                 {"class":"altA"}).find("a").get("href")
            search_result_url = 'https://www.tapology.com/' + search_result_url

            r = requests.get(search_result_url, headers=headers, timeout=20)
            soup = BeautifulSoup(r.text, 'html.parser')

            birth_date_html_elem = soup.find("div", {"id":"stats"}).findAll("li")[4].findAll("span")
            weight_html_elem = soup.find("div", {"id":"stats"}).findAll("li")[6].findAll("span")
            height_reach_html_elem = soup.find("div", {"id":"stats"}).findAll("li")[8].findAll("span")

            #2 means that birth date is known
            if len(birth_date_html_elem) > 1:
                birth_date_tapology = birth_date_html_elem[1].text
                #print(birth_date_tapology)

            if weight_html_elem[1] is not None:
                if weight_html_elem[1].text != 'N/A':
                    weight_tapology = weight_html_elem[1].text +'.'
                    #print(weight_tapology)

            if height_reach_html_elem[0] is not None:
                if height_reach_html_elem[0].text != 'N/A':
                    height_tapology = height_reach_html_elem[0].text.split()[0]
                    #print(height_tapology)

            if height_reach_html_elem[1] is not None:
                if height_reach_html_elem[1].text != 'N/A':
                    reach_tapology = height_reach_html_elem[1].text.split()[0]
                    #print(reach_tapology)


        if birth_date_sherdog is not None:
                birth_date = datetime.strptime(birth_date_sherdog, '%Y-%m-%d').strftime("%b %d, %Y")
        elif birth_date_tapology is not None:
                birth_date = datetime.strptime(birth_date_tapology, '%Y.%m.%d').strftime("%b %d, %Y")

        height = height_sherdog if height_sherdog is not None else height_tapology
        weight = weight_sherdog if weight_sherdog is not None else weight_tapology

        reach = reach_tapology


        if height is not None:
            height = height.split("'")
            height = "' ".join(height)
            df_fighters.iloc[index]['Height'] = height
        if weight is not None:
            df_fighters.iloc[index]['Weight'] = weight
        if reach is not None:
            df_fighters.iloc[index]['Reach'] = reach
        if birth_date is not None:
            df_fighters.iloc[index]['DOB'] = birth_date


    except Exception as e:
        f = open("log/search_errors.txt", "a")
        f.write(row['fighter_name']+";"+first_exception+";"+str(e)+"\n")
        f.close()

    time.sleep(45)
        
df_fighters.to_csv("scraped_fighter_data.csv")
#f.close()

AJ+Fonseca
AJ+Matthews
AJ+McKee
AJ+Siscoe
Aalon+Cruz
Aaron+Brink
Aaron+Ely
Aaron+Jeffery
Aaron+Lanfranco
Aaron+Miller
Aaron+Trujillo
Aaron+Wetherspoon
Aaron+Wilkinson
Abel+Cullum
Abongo+Humphrey
Abram+Torres
Achmed+Labasanov
Adam+Khaliev
Adam+Lynn
Adam+Meredith
Adam+Piccolotti
Adam+Schindler
Adam+Smith
Adam+Steele
Adin+Duenas
Adli+Edwards
Adrian+Diaz
Adrian+Serrano
Adriano+Santos
Adrienna+Jenkins
Aji+Susilo
Akihito+Tanaka
Akira+Kikuchi
Akira+Shoji
Akiyo+Nishiura
Alan+Gomez
Alan+Hiro
Alavutdin+Gadjiev
Albert+Cheng
Albert+Rios
Alberta+Cerra+Leon
Alberto+Crane
Alejandro+Garcia
Alejandro+Villalobos
Aleksa+Camur
Aleksander+Emelianenko
Alex+Andrade
Alex+Crispin
Alex+Dunworth
Alex+Higley
Alex+Huddleston
Alex+Hunter
Alex+Schoenauer
Alex+Stiebling
Alex+Zuniga
Alexander+Crispim
Alexander+Munoz
Alexander+Otsuka
Alexander+Poppeck
Alexander+Romanov
Alexander+Sarnavskiy
Alexander+Shlemenko
Alexander+Trevino
Alexandre+Barros
Alexandre+Dantas
Alexandre+Ferreira
Alexandre+Franca+Nogueira
Alexandru+Lung

In [5]:
df_missing_values = df_fighters.query('Height != Height or Weight != Weight or Reach != Reach or DOB != DOB')
print(len(df_missing_values))

1306
